In [32]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [33]:
df = pd.read_excel('data.xlsx')

In [34]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(df.values)
distances, indices = knn.kneighbors(df.values, n_neighbors=3)

In [35]:
distances

array([[0.00000000e+00, 7.37723105e-03, 9.91490911e-03],
       [0.00000000e+00, 3.18820552e-05, 1.23214028e-02],
       [1.11022302e-16, 1.78231580e-05, 3.97253494e-05],
       [0.00000000e+00, 3.97253494e-05, 4.21869614e-05],
       [1.11022302e-16, 1.57076175e-02, 1.69468038e-02],
       [0.00000000e+00, 1.99133459e-04, 7.37723105e-03],
       [2.22044605e-16, 3.18820552e-05, 1.22725364e-02],
       [0.00000000e+00, 1.78231580e-05, 4.21869614e-05],
       [0.00000000e+00, 1.99133459e-04, 9.91490911e-03]])

In [36]:
# get the index for 'movie_0'
index_for_cluster = df.index.tolist().index(0)
# find the indices for the similar movies
sim_segments = indices[index_for_cluster].tolist()
# distances between 'movie_0' and the similar movies
segment_distances = distances[index_for_cluster].tolist()
# the position of 'movie_0' in the list sim_movies
id_movie = sim_segments.index(index_for_cluster)
# remove 'movie_0' from the list sim_movies
sim_segments.remove(index_for_cluster)
# remove 'movie_0' from the list movie_distances
segment_distances.pop(id_movie)
print('The Nearest cluster to cluster_0:', sim_segments)
print('The Distance from cluster_0:', segment_distances)

The Nearest cluster to cluster_0: [5, 8]
The Distance from cluster_0: [0.007377231045750943, 0.009914909111222281]


In [37]:
df1 = df.copy()

# find the nearest neighbors using NearestNeighbors(n_neighbors=3)
number_neighbors = 3
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(df.values)
distances, indices = knn.kneighbors(df.values, n_neighbors=number_neighbors)

# convert user_name to user_index
user_index = df.columns.tolist().index('TN_Аксессуары')

# t: movie_title, m: the row number of t in df
for m,t in list(enumerate(df.index)):
  
  # find movies without ratings by user_4
  if df.iloc[m, user_index] == 0:
    sim_segments = indices[m].tolist()
    segment_distances = distances[m].tolist()
    
    # Generally, this is the case: indices[3] = [3 6 7]. The movie itself is in the first place.
    # In this case, we take off 3 from the list. Then, indices[3] == [6 7] to have the nearest NEIGHBORS in the list. 
    if m in sim_segments:
      id_movie = sim_segments.index(m)
      sim_segments.remove(m)
      segment_distances.pop(id_movie) 
    else:
      sim_segments = sim_segments[:n_neighbors-1]
      segment_distances = segment_distances[:n_neighbors-1]
        
    # movie_similarty = 1 - movie_distance    
    movie_similarity = [1-x for x in segment_distances]
    movie_similarity_copy = movie_similarity.copy()
    nominator = 0

    # for each similar movie
    for s in range(0, len(movie_similarity)):
      
      # check if the rating of a similar movie is zero
      if df.iloc[sim_segments[s], user_index] == 0:

        # if the rating is zero, ignore the rating and the similarity in calculating the predicted rating
        if len(movie_similarity_copy) == (number_neighbors - 1):
          movie_similarity_copy.pop(s)
          
        else:
          movie_similarity_copy.pop(s-(len(movie_similarity)-len(movie_similarity_copy)))

      # if the rating is not zero, use the rating and similarity in the calculation
      else:
        nominator = nominator + movie_similarity[s]*df.iloc[sim_segments[s],user_index]

    if len(movie_similarity_copy) > 0:
      
        # check if the sum of the ratings of the similar movies is positive.
        if sum(movie_similarity_copy) > 0:
          predicted_r = nominator/sum(movie_similarity_copy)

        # Even if there are some movies for which the ratings are positive, some movies have zero similarity even though they are selected as similar movies.
        # in this case, the predicted rating becomes zero as well  
        else:
          predicted_r = 0

        # if all the ratings of the similar movies are zero, then predicted rating should be zero
    else:
      predicted_r = 0

# place the predicted rating into the copy of the original dataset
    df1.iloc[m,user_index] = predicted_r

In [38]:
def recommend_movies(user, num_recommended_movies):

  print('The list of the Movies {} Has Watched \n'.format(user))

  for m in df[df[user] > 0][user].index.tolist():
    print(m)
  
  print('\n')

  recommended_movies = []

  for m in df[df[user] == 0].index.tolist():

    index_df = df.index.tolist().index(m)
    predicted_rating = df1.iloc[index_df, df1.columns.tolist().index(user)]
    recommended_movies.append((m, predicted_rating))

  sorted_rm = sorted(recommended_movies, key=lambda x:x[1], reverse=True)
  
  print('The list of the Recommended Movies \n')
  rank = 1
  for recommended_movie in sorted_rm[:num_recommended_movies]:
    
    print('{}: {} - predicted rating:{}'.format(rank, recommended_movie[0], recommended_movie[1]))
    rank = rank + 1

In [39]:
recommend_movies('TN_Климат', 5)

The list of the Movies TN_Климат Has Watched 

0
2
3
4
5
6
7


The list of the Recommended Movies 

1: 1 - predicted rating:0.0
2: 8 - predicted rating:0.0


In [40]:
def movie_recommender(user, num_neighbors, num_recommendation):

  number_neighbors = num_neighbors

  knn = NearestNeighbors(metric='cosine', algorithm='brute')
  knn.fit(df.values)
  distances, indices = knn.kneighbors(df.values, n_neighbors=number_neighbors)

  user_index = df.columns.tolist().index(user)

  for m,t in list(enumerate(df.index)):
    if df.iloc[m, user_index] == 0:
      sim_movies = indices[m].tolist()
      movie_distances = distances[m].tolist()
    
      if m in sim_movies:
        id_movie = sim_movies.index(m)
        sim_movies.remove(m)
        movie_distances.pop(id_movie) 

      else:
        sim_movies = sim_movies[:n_neighbors-1]
        movie_distances = movie_distances[:n_neighbors-1]
           
      movie_similarity = [1-x for x in movie_distances]
      movie_similarity_copy = movie_similarity.copy()
      nominator = 0

      for s in range(0, len(movie_similarity)):
        if df.iloc[sim_movies[s], user_index] == 0:
          if len(movie_similarity_copy) == (number_neighbors - 1):
            movie_similarity_copy.pop(s)
          
          else:
            movie_similarity_copy.pop(s-(len(movie_similarity)-len(movie_similarity_copy)))
            
        else:
          nominator = nominator + movie_similarity[s]*df.iloc[sim_movies[s],user_index]
          
      if len(movie_similarity_copy) > 0:
        if sum(movie_similarity_copy) > 0:
          predicted_r = nominator/sum(movie_similarity_copy)
        
        else:
          predicted_r = 0

      else:
        predicted_r = 0
        
      df1.iloc[m,user_index] = predicted_r
  recommend_movies(user, num_recommendation)

In [41]:
movie_recommender('TN_Автотовары', 3, 4)

The list of the Movies TN_Автотовары Has Watched 

1
5


The list of the Recommended Movies 

1: 0 - predicted rating:0.002622377622377622
2: 8 - predicted rating:0.002622377622377622
3: 6 - predicted rating:0.0009772142244255887
4: 2 - predicted rating:0.0


In [42]:
movie_recommender('TN_Автотовары', 3, 4)

The list of the Movies TN_Автотовары Has Watched 

1
5


The list of the Recommended Movies 

1: 0 - predicted rating:0.002622377622377622
2: 8 - predicted rating:0.002622377622377622
3: 6 - predicted rating:0.0009772142244255887
4: 2 - predicted rating:0.0


In [44]:
movie_recommender('TN_Аксессуары', 3, 4)

The list of the Movies TN_Аксессуары Has Watched 

0
1
2
3
4
5
6
7
8


The list of the Recommended Movies 



In [45]:
movie_recommender('TN_Детские товары', 3, 4)

The list of the Movies TN_Детские товары Has Watched 

0
1
2
3
4
5
7


The list of the Recommended Movies 

1: 8 - predicted rating:0.4828451600481376
2: 6 - predicted rating:0.137715702164172


In [46]:
movie_recommender('TN_Игры, софт и развлечения', 3, 4)

The list of the Movies TN_Игры, софт и развлечения Has Watched 

0
1
2
3
5
6
8


The list of the Recommended Movies 

1: 7 - predicted rating:1.0625114208755166
2: 4 - predicted rating:0.125


In [47]:
movie_recommender('TN_Мебель', 3, 4)

The list of the Movies TN_Мебель Has Watched 

0
1
3
4
6
7


The list of the Recommended Movies 

1: 8 - predicted rating:1.3553251649387372
2: 5 - predicted rating:1.355325164938737
3: 2 - predicted rating:1.187508898021298


In [48]:
movie_recommender('TN_Мелкая бытовая техника', 3, 4)

The list of the Movies TN_Мелкая бытовая техника Has Watched 

0
1
2
3
4
5
7
8


The list of the Recommended Movies 

1: 6 - predicted rating:0.49604149752609195


In [50]:
movie_recommender('TN_Товары для дома', 3, 4)

The list of the Movies TN_Товары для дома Has Watched 

0
1
2
3
4
5
7
8


The list of the Recommended Movies 

1: 6 - predicted rating:0.14540934158378765


In [53]:
movie_recommender('TN_Цифровая Техника', 3, 4)

The list of the Movies TN_Цифровая Техника Has Watched 

0
2
3
4
5
6
7
8


The list of the Recommended Movies 

1: 1 - predicted rating:0.2694209790440791


In [ ]:
movie_recommender('TN_Цифровая Техника', 3, 4)

In [54]:
movie_recommender('TN_Элитная техника', 3, 4)

The list of the Movies TN_Элитная техника Has Watched 

0


The list of the Recommended Movies 

1: 5 - predicted rating:0.002827521206409048
2: 8 - predicted rating:0.002827521206409048
3: 1 - predicted rating:0.0
4: 2 - predicted rating:0.0


In [43]:
#['TN_Автотовары', 'TN_Аксессуары', 'TN_Детские товары',
 #      'TN_Игры, софт и развлечения', 'TN_Климат',
  #     'TN_Крупная бытовая техника', 'TN_Мебель', 'TN_Мелкая бытовая техника',
   #    'TN_Товары для дома', 'TN_Услуги', 'TN_Хобби, досуг',
  #     'TN_Цифровая Техника', 'TN_Элитная техника']